# CyprusDB generation

This notebook contains all the steps followed to generate the CyprusDB dataset. This ensures its reproducibility.

## Data sources

1) [CyStat's 2011 census](https://www.data.gov.cy/dataset/%CF%80%CE%BB%CE%B7%CE%B8%CF%85%CF%83%CE%BC%CF%8C%CF%82-%CE%BA%CE%B1%CF%84%CE%AC-%CF%84%CF%8C%CF%80%CE%BF-%CE%B4%CE%B9%CE%B1%CE%BC%CE%BF%CE%BD%CE%AE%CF%82-%CE%B1%CF%80%CE%BF%CE%B3%CF%81%CE%B1%CF%86%CE%AE-%CF%80%CE%BB%CE%B7%CE%B8%CF%85%CF%83%CE%BC%CE%BF%CF%8D-2011)
2) [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/overview?hl=en-419)

# Preparation

In [27]:
### Imports ###

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import missingno as msno
import folium

# Data retrieval
import requests
import json

# Data reading
import tabula

# Others
import time
from pprint import pprint
import os
import glob
from datetime import date

# 1) Population list

Source: [CyStat](https://www.data.gov.cy/dataset/%CF%80%CE%BB%CE%B7%CE%B8%CF%85%CF%83%CE%BC%CF%8C%CF%82-%CE%BA%CE%B1%CF%84%CE%AC-%CF%84%CF%8C%CF%80%CE%BF-%CE%B4%CE%B9%CE%B1%CE%BC%CE%BF%CE%BD%CE%AE%CF%82-%CE%B1%CF%80%CE%BF%CE%B3%CF%81%CE%B1%CF%86%CE%AE-%CF%80%CE%BB%CE%B7%CE%B8%CF%85%CF%83%CE%BC%CE%BF%CF%8D-2011)

The base town list is extracted from the above link. The list is then cleaned and the population is extracted from the excel file. The complete list of settlements used in this project is teken from the sheet `Γ2`.

In [2]:
# Open raw file
filepath = 'sources/Census 2011 Excel format/POP_CEN_11-POP_PLACE_RESID-EL-171115.xls'

# Read file
raw_census_df = pd.read_excel(filepath, sheet_name='Γ2', skiprows=3)
raw_census_df.head()

,Unnamed: 0,ΓΕΩΓ/ΚΟΣ ΚΩΔΙΚΟΣ,"ΕΠΑΡΧΙΑ, ΔΗΜΟΣ/ΚΟΙΝΟΤΗΤΑ ΚΑΙ ΕΝΟΡΙΑ",ΝΟΙΚΟΚΥΡΙΑ,Unnamed: 4,Unnamed: 5,Unnamed: 6,ΙΔΡΥΜΑΤΑ,Unnamed: 8,Unnamed: 9,Unnamed: 10,ΣΥΝΟΛΟ ΠΛΗΘΥΣΜΟΥ,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,ΑΡΙΘΜΟΣ,ΠΛΗΘΥΣΜΟΣ,NaN,NaN,ΑΡΙΘΜΟΣ,ΠΛΗΘΥΣΜΟΣ,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Σύνολο,Άνδρες,Γυναίκες,NaN,Σύνολο,Άνδρες,Γυναίκες,Σύνολο,Άνδρες,Γυναίκες
2,NaN,NaN,Σύνολο,303242,836566,407228,429338,211,3841,1552,2289,840407,408780,431627
3,NaN,1,Επαρχία Λευκωσίας,119203,324952,157307,167645,94,2028,955,1073,326980,158262,168718
4,NaN,1000,Δήμος Λευκωσίας,22833,54452,26086,28366,11,562,434,128,55014,26520,28494


## Data cleaning

In [3]:
# Retain only relevant columns
to_retain = [
    'ΓΕΩΓ/ΚΟΣ ΚΩΔΙΚΟΣ',
    'ΕΠΑΡΧΙΑ, ΔΗΜΟΣ/ΚΟΙΝΟΤΗΤΑ ΚΑΙ ΕΝΟΡΙΑ',
    'ΣΥΝΟΛΟ ΠΛΗΘΥΣΜΟΥ',
    'Unnamed: 12', # Males
    'Unnamed: 13' # Fermales
]

census_df = raw_census_df[to_retain].copy()

# Rename columns
to_rename = {
    'ΓΕΩΓ/ΚΟΣ ΚΩΔΙΚΟΣ': 'geo_code',
    'ΕΠΑΡΧΙΑ, ΔΗΜΟΣ/ΚΟΙΝΟΤΗΤΑ ΚΑΙ ΕΝΟΡΙΑ': 'town',
    'ΣΥΝΟΛΟ ΠΛΗΘΥΣΜΟΥ': 'population',
    'Unnamed: 12' : 'male_population',
    'Unnamed: 13' : 'female_population'
}

census_df.rename(columns=to_rename, inplace=True)

# Drop the three first rows for format purposes
census_df.drop([0, 1, 2], inplace=True)

# Drop the last six rows, which provide no information
census_df.drop(census_df.tail(6).index, inplace=True)

# Fill NaNs with 0 in male and female population columns
census_df.fillna(value = {'male_population' : 0, 'female_population' : 0}, inplace=True)

census_df.head()

,geo_code,town,population,male_population,female_population
3,1,Επαρχία Λευκωσίας,326980,158262,168718
4,1000,Δήμος Λευκωσίας,55014,26520,28494
5,100001,Άγιος Ανδρέας,5767,2817,2950
6,100002,Τρυπιώτης,2158,983,1175
7,100003,Νεμπέτ Χανέ,189,86,103


### Create the 'district' column

In [4]:
# Extract the district from the 'town' column and create a new column
# if 'Επαρχία' is in the town name, then the district is the town name
census_df['district'] = census_df['town'].apply(lambda x: x.split(' ')[1] if 'Επαρχία' in x else np.nan)

# Fill the NaN values with the previous value
# using a forward fill
census_df['district'].fillna(method='ffill', inplace=True)

# Remove the rows that contain the district name
census_df = census_df[~census_df['town'].str.contains('Επαρχία')]

census_df.head()

,geo_code,town,population,male_population,female_population,district
4,1000,Δήμος Λευκωσίας,55014,26520,28494,Λευκωσίας
5,100001,Άγιος Ανδρέας,5767,2817,2950,Λευκωσίας
6,100002,Τρυπιώτης,2158,983,1175,Λευκωσίας
7,100003,Νεμπέτ Χανέ,189,86,103,Λευκωσίας
8,100004,Ταμπάκ Χανέ,299,117,182,Λευκωσίας


### Include suburbs into the main cities
This requieres reversing the genitives of the dimos to the nominative case. Since the number of suburbs is small, we can do this manually


In [5]:
# Map genitive town names to nominative town names
dimos_manual_mapping = {
    # Nicosia
    'Δήμος Λευκωσίας' : 'Λευκωσία',
    'Δήμος Αγίου Δομετίου' : 'Άγιος Δομέτιος',
    'Δήμος Έγκωμης' : 'Έγκωμη',
    'Δήμος Στροβόλου' : 'Στροβόλος',
    'Δήμος Αγλαντζιάς' : 'Αγλαντζιά',
    'Δήμος Λακατάμειας' : 'Λακατάμεια',
    'Δήμος Λατσιών' : 'Λατσιά',
    'Δήμος Ιδαλίου' : 'Δάλι',
    # Larnaka
    'Δήμος Λάρνακας' : 'Λάρνακα',
    'Δήμος Αραδίππου' : 'Αραδίππου',
    # Limassol
    'Δήμος Λεμεσού' : 'Λεμεσός',
    'Δήμος Μέσα Γειτονιάς' : 'Μέσα Γειτονιά',
    'Δήμος Αγίου Αθανασίου' : 'Άγιος Αθανάσιος',
    'Δήμος Γερμασόγειας' : 'Γερμασόγεια',
    'Δήμος Κάτω Πολεμιδιών' : 'Κάτω Πολεμίδια',
    # Paphos
    'Δήμος Πάφου' : 'Πάφος',
    'Δήμος Γεροσκήπου' : 'Γεροσκήπου',
    'Δήμος Πόλεως Χρυσοχούς' : 'Πόλις Χρυσοχούς',
    'Δήμος Πέγειας' : 'Πέγεια'
}

# Create a column indicating whether the town name is a dimos
census_df['is_dimos'] = census_df['town'].apply(lambda x: True if x in dimos_manual_mapping.keys() else False)

# Replace dimos names with settlement names
census_df['town'] = census_df['town'].apply(lambda x: dimos_manual_mapping[x] if x in dimos_manual_mapping.keys() else x)

In [6]:
census_df.head(5)

,geo_code,town,population,male_population,female_population,district,is_dimos
4,1000,Λευκωσία,55014,26520,28494,Λευκωσίας,True
5,100001,Άγιος Ανδρέας,5767,2817,2950,Λευκωσίας,False
6,100002,Τρυπιώτης,2158,983,1175,Λευκωσίας,False
7,100003,Νεμπέτ Χανέ,189,86,103,Λευκωσίας,False
8,100004,Ταμπάκ Χανέ,299,117,182,Λευκωσίας,False


## Remove suburbs from locations

This is done for several reasons:
- Have a consistent data structure: all locations are settlements, not suburbs
- Avoid double population counts

This behaviour can be switched off by setting `remove_suburbs=False`

In [7]:
remove_suburbs = True

# Remove suburbs
# The suburbs are the towns that have a six-digit geo code
if remove_suburbs:
    census_df = census_df[census_df['geo_code'].apply(lambda x: len(str(x)) < 6)].reset_index(drop=True)

In [8]:
census_df.head()

,geo_code,town,population,male_population,female_population,district,is_dimos
0,1000,Λευκωσία,55014,26520,28494,Λευκωσίας,True
1,1010,Άγιος Δομέτιος,12456,5861,6595,Λευκωσίας,True
2,1011,Έγκωμη,18010,8547,9463,Λευκωσίας,True
3,1012,Στροβόλος,67904,32248,35656,Λευκωσίας,True
4,1013,Αγλαντζιά,20783,9803,10980,Λευκωσίας,True


In [9]:
# Save a checkpoint of the dataframe to a csv file
version = 1

census_df.to_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv', index=False)

In [10]:
# Load the checkpoint
census_df = pd.read_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv')

# 2) Google Maps API (Geocoding API)

Source: [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/overview?hl=en-419)

## Notes:
- The API key is stored in a file called `api_key.txt`. This file is not included in the repository. To reproduce the results here, you need to create your own API key and store it in a file called `api_key.txt` in the folder `private_utils`.
- Requests are limited to 2500 per day. 
- Retrieving data from the API is not free. Be aware of the costs when rebuilding the dataset.

In [11]:
# Read API key
with open('private_utils/api_key.txt', 'r') as f:
    api_key = f.read()

## Auxiliary functions

In [113]:
# Extract coordinates for towns in Cyprus from Geocoding API
def extract_coordinates(
        town: str,
        district: str = None,
        api_key: str = api_key, 
        boundaries: list = [34.51, 32.17, 35.73, 34.61]) -> tuple:
    """
    Extract coordinates for a town in Cyprus from Google Maps Geocoding API.

    The boundaries argument 

    Parameters
    ----------
    town : str
        The town name.
    district : str
        The district name.
    api_key : str
        The Google Maps API key.
    boundaries : list
        The boundaries of the search area. It is a list of four floats that represent the
        boundaries of the search area. The order of the floats is as follows:
        [southwest_lat, southwest_lon, northeast_lat, northeast_lon].
        By default, the boundaries are set to the approximate boundaries of Cyprus island.

    Returns
    -------
    lat : float
        The latitude of the town.
    lon : float
        The longitude of the town.
    gm_id : str
        The Google Maps ID of the town.
    """
        
    # Set search term
    query = town

    # Add district to search term
    if district is not None:
        query += ' ' + district

    # Set Geocoding API URL
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address=' + query + '&key=' + api_key

    # Add boundaries to search
    if boundaries:
        url += '&bounds=' + str(boundaries[0]) + ',' + str(boundaries[1]) + '|' + str(boundaries[2]) + ',' + str(boundaries[3])

    # Extract coordinates
    response = requests.get(url)
    data = json.loads(response.text)
    
    # Extract coordinates and Google Maps ID
    lat = data['results'][0]['geometry']['location']['lat']
    lon = data['results'][0]['geometry']['location']['lng']
    gm_id = data['results'][0]['place_id']

    return lat, lon, gm_id

## Data retrieval

In [13]:
census_df.head()

,geo_code,town,population,male_population,female_population,district,is_dimos
0,1000,Λευκωσία,55014,26520,28494,Λευκωσίας,True
1,1010,Άγιος Δομέτιος,12456,5861,6595,Λευκωσίας,True
2,1011,Έγκωμη,18010,8547,9463,Λευκωσίας,True
3,1012,Στροβόλος,67904,32248,35656,Λευκωσίας,True
4,1013,Αγλαντζιά,20783,9803,10980,Λευκωσίας,True


In [14]:
# Generate coordinates if requested
generate_coordinates = True

if generate_coordinates:
    # Extract coordinates for towns in Cyprus from Geocoding API
    # Takes ~ 1 minute
    # Latitude, longitude, Google Maps ID
    census_df['lat'], census_df['lon'], census_df['gm_id'] = zip(*census_df.apply(lambda x: extract_coordinates(x['town'], distrinct = x['district']), axis=1))

    # Save coordinates with retrieval date
    census_coordinates = census_df[['town', 'district', 'lat', 'lon', 'gm_id']]
    census_coordinates.to_csv(f'sources/Geocoding API/geocoding_coordinates_ROC_{date.today().strftime("%Y-%m-%d")}.csv', index=False)

else:
    # Select the latest coordinates file
    list_of_files = glob.glob('sources/Geocoding API/geocoding_coordinates_ROC_*.csv')
    latest_file = max(list_of_files, key=os.path.getctime)
    
    # Load coordinates from file
    census_coordinates = pd.read_csv(latest_file)

# Add coordinates to census dataframe
# census_df = census_df.merge(census_coordinates, on=['town', 'district'], how='left')
census_df[['lat', 'lon', 'gm_id']] = census_coordinates[['lat', 'lon', 'gm_id']]

census_df.head()

,geo_code,town,population,male_population,female_population,district,is_dimos,lat,lon,gm_id
0,1000,Λευκωσία,55014,26520,28494,Λευκωσίας,True,35.185566,33.382276,ChIJVU1JymcX3hQRpcARA5ykXls
1,1010,Άγιος Δομέτιος,12456,5861,6595,Λευκωσίας,True,35.172787,33.329092,ChIJqzUpSaEQ3hQRgmUX_emhREA
2,1011,Έγκωμη,18010,8547,9463,Λευκωσίας,True,35.153823,33.316954,ChIJtcdnJb8Q3hQR80ccwpjkIPk
3,1012,Στροβόλος,67904,32248,35656,Λευκωσίας,True,35.143663,33.343791,ChIJ90x2dika3hQRq7-H2HRHAJo
4,1013,Αγλαντζιά,20783,9803,10980,Λευκωσίας,True,35.149803,33.394086,ChIJb0vkuNMZ3hQR_4oSWBdFjX0


## Plot and inspect results

In [19]:
# Show the retrieval results to check for errors
# Plot all towns in Cyprus
map = folium.Map(location=[35.1264, 33.4299], zoom_start=9)

for i in range(len(census_df)):
    folium.CircleMarker(
        location=[census_df['lat'][i], census_df['lon'][i]],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map)
    
map

## Results notes

1) The coordinates for the town of Πόλη Χρυσοχούς (Póli Chrysochoús) are not correct. This is due to conflicting names with other natural places and locations and to the fact that the algorithm only takes the first result. Since it is the only instance of such a problem, a manual correction is applied.

In [20]:
# Prepare manual corrections 
coordinates_manual_mapping = {
    'Πόλις Χρυσοχούς' : [35.0339441, 32.4253751, 'ChIJG4zmlxR05xQR7lc0vj1h-YQ']
}

# Apply manual corrections
for town in coordinates_manual_mapping.keys():
    census_df.loc[census_df['town'] == town, ['lat', 'lon', 'gm_id']] = coordinates_manual_mapping[town]

## Inspect results after correction

In [21]:
map = folium.Map(location=[35.1264, 33.4299], zoom_start=9)

for i in range(len(census_df)):
    folium.CircleMarker(
        location=[census_df['lat'][i], census_df['lon'][i]],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map)
    
map

## Save checkpoint

In [22]:
# Save a checkpoint of the dataframe to a csv file
version = 2

census_df.to_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv', index=False)
# Load the checkpoint
census_df = pd.read_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv')

In [26]:
# Load the checkpoint
census_df = pd.read_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv')

# 3) Add census data for Northern Cyprus

Source: [2011 Census of Northern Cyprus](https://www.ktoeos.org/wp-content/uploads/2013/08/nufus_ikinci_.pdf)

The data is extracted from the above PDF. Target table is table 5, which contains population information for each town for years 2006 and 2011. Pages 25 to 33 contain the data. 

## Read pages

In [87]:
# Read each individual page from the PDF file
# The tables are in pages 25 to 33
pages = tabula.read_pdf('sources/TRNC Census 2011/nufus_ikinci_.pdf', pages='25-33')

# Delete the three first rows from every page
# They contain general information
for page in pages:
    page.drop(page.index[:3], inplace=True)

## Clean pages

In [88]:
# Concatenate all pages into a single dataframe
census_df_trnc = pd.concat(pages, ignore_index=True)

# Rename columns
# 'mixed_data' contains several columns together due to OCR errors 
column_names = ['town', 'population_2006', 'mixed_data', 'male_population', 'female_population']
census_df_trnc.columns = column_names

# Separete the columns in 'mixed_data'
new_columns = ['male_population_2006', 'female_population_2006', 'population']
census_df_trnc[new_columns] = census_df_trnc['mixed_data'].str.split(' ', expand=True)

# Reorder columns
order = ['town', 
         'population', 'male_population', 'female_population', 
         'population_2006', 'male_population_2006', 'female_population_2006']
census_df_trnc = census_df_trnc[order]

# Remove rows where the town is not specified
# Those rows are headers
census_df_trnc = census_df_trnc[census_df_trnc['town'].notna()]

census_df_trnc.head()

,town,population,male_population,female_population,population_2006,male_population_2006,female_population_2006
1,Lefkoşa İlçe Toplamı,"94,824","49,838","44,986","84,776","46,187","38,589"
2,Lefkoşa Merkez Bucak Toplamı,"82,929","43,628","39,301","72,479","39,337","33,142"
3,Lefkoşa Belediye Toplamı,"61,378","32,260","29,118","56,146","30,583","25,563"
4,ABDİ ÇAVUŞ,568,315,253,975,591,384
5,AKKAVUK,793,458,335,898,498,400


## Extract districts, subdistricts and municipalities

In [89]:
# Create a columns named 'district'
# If 'İlçe' ('district') is in the town name, extract it
census_df_trnc['district'] = census_df_trnc['town'].apply(lambda x: x.split(' İlçe ')[0] if ' İlçe' in x else np.nan)

# Apply a forward fill to fill the missing values
census_df_trnc['district'] = census_df_trnc['district'].ffill()

In [90]:
# Create a column for subdistricts
# If 'Bucak' ('subdistrict') is in the town name, extract it
census_df_trnc['subdistrict'] = census_df_trnc['town'].apply(lambda x: x.split(' ')[0] if ' Bucak' in x else np.nan)

# Apply a forward fill to fill the missing values
census_df_trnc['subdistrict'] = census_df_trnc['subdistrict'].ffill() 

In [91]:
# Create a column for municipalities
# If 'Belediye' ('municipality') is in the town name, extract it
census_df_trnc['municipality'] = census_df_trnc['town'].apply(lambda x: x.split(' ')[0] if ' Belediye' in x else np.nan)

# Apply a forward fill to fill the missing values
census_df_trnc['municipality'] = census_df_trnc['municipality'].ffill()

## Final cleaning

In [103]:
# Remove rows including 'Toplamı' ('Total')
census_df_trnc = census_df_trnc.loc[~census_df_trnc['town'].str.contains('Toplamı')]

### Lowercase town names

We can not apply a simple lowercase function, Since 'ı' is merged with 'i'. Consequently, we need to first replace 'I' with 'ı' and then lowercase the whole string.


In [110]:
def lowercase_turkish_town_name(town: str) -> str:
    """Lowercases the Turkish town names in the dataframe.

    Parameters
    ----------
    town : str
        The town name to be lowercased.

    Returns
    -------
    str
        The lowercased town name.

    """

    # Split the town name into words
    words = town.split(' ')
    
    processed_words = []
    for word in words:
        # Separate word parts
        initial = word[0]
        body = word[1:]

        # Replace 'I' with 'ı' in the body, then lowercase it
        body = body.replace('I', 'ı').lower()

        # Join the word parts back together
        word = initial + body

        processed_words.append(word)
        

    # Join the words back into a string
    town = ' '.join(processed_words)

    return town

In [111]:
# Apply lowercasing function
census_df_trnc['town'] = census_df_trnc['town'].apply(lowercase_turkish_town_name)

## Save checkpoint

In a secondary folder prior to merger with the main dataset

In [115]:
# Save a checkpoint of the dataframe to a csv file
version = 1

census_df_trnc.to_csv(f'checkpoints/secondary_checkpoints/TRNC_census_cp_v{str(version)}.csv', index=False)

In [116]:
# Load the checkpoint
census_df_trnc = pd.read_csv(f'checkpoints/secondary_checkpoints/TRNC_census_cp_v{str(version)}.csv')

# 4) Google Maps API for northern Cyprus

### Important notes

Data for settlements in northern Cyprus is heavily unreliable: the coordinates offered point to the center of the administrative unit and not to the settlement itself. Altough the coordinates can be obtained through OpenStreetMap API, the Google Maps API is still needed to get consistent Greek and Turkish namings for towns. As such, coordinates will be generated and will be overwritten by OpenStreetMap data later on.

In [201]:
# Extract coordinates for towns in Cyprus from Geocoding API
def extract_coordinates(
        town: str,
        district: str = None,
        language: str = None,
        api_key: str = api_key, 
        boundaries: list = [34.51, 32.17, 35.73, 34.61]) -> tuple:
    """
    Extract coordinates for a town in Cyprus from Google Maps Geocoding API.

    The boundaries argument 

    Parameters
    ----------
    town : str
        The town name.
    district : str
        The district name.
    api_key : str
        The Google Maps API key.
    boundaries : list
        The boundaries of the search area. It is a list of four floats that represent the
        boundaries of the search area. The order of the floats is as follows:
        [southwest_lat, southwest_lon, northeast_lat, northeast_lon].
        By default, the boundaries are set to the approximate boundaries of Cyprus island.

    Returns
    -------
    lat : float
        The latitude of the town.
    lon : float
        The longitude of the town.
    gm_id : str
        The Google Maps ID of the town.
    """
        
    # Set search term
    query = town

    # Add district to search term
    if district is not None:
        query += ' ' + district
        print(query)
   # 

    # Set Geocoding API URL
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address=' + query + '&key=' + api_key

    # Add boundaries to search
    if boundaries:
        url += '&bounds=' + str(boundaries[0]) + ',' + str(boundaries[1]) + '|' + str(boundaries[2]) + ',' + str(boundaries[3])
        print('&bounds=' + str(boundaries[0]) + ',' + str(boundaries[1]) + '|' + str(boundaries[2]) + ',' + str(boundaries[3]))
    
    # Add language
    if language is not None:
        query += ' ' + language

    ### Request execution ###

    # Extract coordinates
    response = requests.get(url)
    data = json.loads(response.text)
    
    # Extract coordinates and Google Maps ID
    try:
        
        # If there are no results with 'locality', return NaNs
        if len(data['results']) == 0:
            print('Unable to extract coordinates for ' + town + ': no results found.')
            return np.nan, np.nan, np.nan

        # Get the first results that contains 'locality' in the types list and
        # is within the boundaries
        valid_i = None
        for i, result in enumerate(data['results']):
            # Check that the result contains 'locality'
            if 'locality' in result['types']:
                # Check that the result is within the boundaries
                lat = data['results'][i]['geometry']['location']['lat']
                lon = data['results'][i]['geometry']['location']['lng']
                if lat < boundaries[0] or lat > boundaries[2] or lon < boundaries[1] or lon > boundaries[3]:
                    continue

                # If all checks are valid, set the index of the result
                valid_i = i
                break

            # If there are no results, return NaNs
            elif i == len(data['results']) - 1:
                print('Unable to extract coordinates for ' + town + ': no locality results found.')
                return np.nan, np.nan, np.nan
            
        # If there are no valid results, return NaNs
        if valid_i is None:
            print('Unable to extract coordinates for ' + town + ': no valid results found.')
            return np.nan, np.nan, np.nan

        # Extract coordinates and Google Maps ID
        lat = data['results'][valid_i]['geometry']['location']['lat']
        lon = data['results'][valid_i]['geometry']['location']['lng']
        gm_id = data['results'][valid_i]['place_id']

    except ValueError:
        print('Unable to extract coordinates for ' + town + '. General error')
        lat = np.nan
        lon = np.nan
        gm_id = np.nan

    return lat, lon, gm_id

In [ ]:
# Generate coordinates if requested
generate_coordinates = True

if generate_coordinates:
    # Extract coordinates for towns in Cyprus from Geocoding API
    # Takes ~ 1 minute
    # Latitude, longitude, Google Maps ID
    census_df_trnc['lat'], census_df_trnc['lon'], census_df_trnc['gm_id'] = zip(*census_df_trnc.apply(lambda x: extract_coordinates(x['town'], 
                                                        district = 'Kıbrıs',
                                                        boundaries=[34.9, 32.5, 35.8, 34.6]), axis=1))

    # Save coordinates with retrieval date
    census_coordinates = census_df_trnc[['town', 'district', 'lat', 'lon', 'gm_id']]
    census_coordinates.to_csv(f'sources/Geocoding API/geocoding_coordinates_TRNC_{date.today().strftime("%Y-%m-%d")}.csv', index=False)

else:
    # Select the latest coordinates file
    list_of_files = glob.glob('sources/Geocoding API/geocoding_coordinates_TRNC_*.csv')
    latest_file = max(list_of_files, key=os.path.getctime)
    
    # Load coordinates from file
    census_coordinates = pd.read_csv(latest_file)

# Add coordinates to census dataframe
# census_df = census_df.merge(census_coordinates, on=['town', 'district'], how='left')
census_df_trnc[['lat', 'lon', 'gm_id']] = census_df_trnc[['lat', 'lon', 'gm_id']]

census_df_trnc.head()

In [203]:
## Plot and inspect results
# Show the retrieval results to check for errors
# Plot all towns in Cyprus
map = folium.Map(location=[35.1264, 33.4299], zoom_start=9)

for i in range(len(census_df_trnc)):
    try:
        folium.CircleMarker(
            location=[census_df_trnc['lat'][i], census_df_trnc['lon'][i]],
            radius=5,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False).add_to(map)
    except ValueError:
        pass
    
map

In [204]:
# Search results for Inönü
census_df_trnc

,town,population,male_population,female_population,population_2006,male_population_2006,female_population_2006,district,subdistrict,municipality,lat,lon,gm_id
0,Abdi̇ Çavuş,568,315,253,975,591,384,Lefkoşa,Lefkoşa,Lefkoşa,NaN,NaN,NaN
1,Akkavuk,793,458,335,898,498,400,Lefkoşa,Lefkoşa,Lefkoşa,NaN,NaN,NaN
2,Arabahmet,561,297,264,761,425,336,Lefkoşa,Lefkoşa,Lefkoşa,NaN,NaN,NaN
3,Aydemet,"2,314","1,147","1,167","1,550",765,785,Lefkoşa,Lefkoşa,Lefkoşa,35.172787,33.329092,ChIJqzUpSaEQ3hQRgmUX_emhREA
4,Ayyıldız,489,271,218,559,316,243,Lefkoşa,Lefkoşa,Lefkoşa,NaN,NaN,NaN
5,Çağlayan,"1,307",667,640,"1,413",744,669,Lefkoşa,Lefkoşa,Lefkoşa,NaN,NaN,NaN
6,Göçmenköy,"3,003","1,551","1,452","2,946","1,526","1,420",Lefkoşa,Lefkoşa,Lefkoşa,NaN,NaN,NaN
7,Hami̇tköy,"5,338","2,773","2,565","2,898","1,567","1,331",Lefkoşa,Lefkoşa,Lefkoşa,35.221519,33.376667,ChIJC26FTOsW3hQRKlGoqkbKgVU
8,Haspolat,"4,204","2,385","1,819","3,380","2,168","1,212",Lefkoşa,Lefkoşa,Lefkoşa,35.210724,33.427140,ChIJ0WZmiRQW3hQRWJCZ8IphTlo
9,Haydarpaşa,155,80,75,320,186,134,Lefkoşa,Lefkoşa,Lefkoşa,NaN,NaN,NaN
